In [2]:
from evaluator import GSM8kEvaluator
import re

In [3]:
evaluator = GSM8kEvaluator()

In [17]:
completion: str = '<reasoning>\nWeng earns $12 an hour for babysitting, and there are 60 minutes in an hour. \nShe just did 50 minutes of babysitting. \nTo calculate her earnings, we can divide the total time she worked by the average rate per hour: 50 minutes / $12/ hour = 4.1667 hours.\nSince she earns $12 an hour, to find her earnings for the 4.1667 hours, we multiply the number of hours by her hourly rate: 4.1667 hours * $12/hour = $49.996.\nSo, Weng earned $49.996 yesterday.\n</reasoning>\n<answer>\nWeng earned $49.996 from babysitting yesterday.'

In [18]:
formatted_completions = [[{"content": completion}]]

In [24]:
# Format the completion in the expected structure
evaluator._xml_count_reward(formatted_completions)

[-0.15500000000000003]

In [25]:
# try other ones
evaluator._soft_format_reward(formatted_completions)

[0.0]

In [20]:
# modified completion
modified_completion: str = '<reasoning>\nWeng earns $12 an hour for babysitting, and there are 60 minutes in an hour. \nShe just did 50 minutes of babysitting. \nTo calculate her earnings, we can divide the total time she worked by the average rate per hour: 50 minutes / $12/ hour = 4.1667 hours.\nSince she earns $12 an hour, to find her earnings for the 4.1667 hours, we multiply the number of hours by her hourly rate: 4.1667 hours * $12/hour = $49.996.\nSo, Weng earned $49.996 yesterday.\n</reasoning>\n<answer>\nWeng earned $49.996 from babysitting yesterday.\n</answer>\n'
formatted_modified_completions = [[{"content": modified_completion}]]

In [22]:
# Format the completion in the expected structure
evaluator._xml_count_reward(formatted_modified_completions)

[0.5]

In [23]:
evaluator._soft_format_reward(formatted_modified_completions)

[0.0]

In [26]:
evaluator._strict_format_reward(formatted_modified_completions)

[0.0]

Still fails -- due to newlines!

In [35]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [36]:
SYSTEM_PROMPT

'\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n'

In [34]:
# Try relaxed versions allowing newlines!
def new_strict_format_reward(completions) -> list[float]:
    """Reward for strict XML format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.match(pattern, r, flags=re.DOTALL)) for r in responses]
    return [0.5 if m else 0.0 for m in matches]



def new_soft_format_reward(completions) -> list[float]:
    """Reward for relaxed XML format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.search(pattern, r, flags=re.DOTALL)) for r in responses]
    return [0.5 if m else 0.0 for m in matches]



In [37]:
new_soft_format_reward(formatted_modified_completions)

[0.5]

In [38]:
new_strict_format_reward(formatted_modified_completions)

[0.5]

Test the new extractor on ints

In [3]:
[[{"content": " 5"}]]

[[{'content': ' 5'}]]

In [4]:
evaluator._int_format_reward([[{"content": " 5"}]])

[0.0]

In [5]:
evaluator._correctness_reward(None, [[{"content": " 5"}]], '5')

[0.0]

In [ ]:
evaluator._int_format_reward([[{"content": " 5"}]])

An example to investigate

In [1]:
completion = "<reasoning>\nTo find out how long Tony takes to assemble a widget, we need to first determine how many widgets are made in a 8-hour shift, which equals to 48 minutes.\n\nSam and Jack together can assemble 2 widgets every 15 minutes, so in 6 hours (48 minutes), they can assemble:\n\n2 * (6/15 * 48) = 8 * 48 = 384 widgets.\n\nThey work together for 4 hours (48 minutes), so in these 4 hours they can assemble:\n\n2 * (4/15 * 48) = 8 * 48 = 384 widgets.\n\nTogether, Sam and Jack together can assemble 384 widgets in 8 hours.\n\nSo, the time it takes for Jack to assemble 2 widgets (equivalent to the 384 widgets made by Sam in 4 hours) can be found by:\n\n384 * 15 minutes (as it's 15 minutes per widget), or:\n\n384 * 45 minutes (as it's 8 hours divided by 6 hours: 45 minutes per widget).\n\nTo find out how long it takes Tony to assemble 1 widget (which equals to 5 minutes), we first find out how many widgets are made by the time he finishes assembling 1 widget:\n\n(8/15 * 45) widgets = 12 widgets (as 8 hours is 45 minutes divided by 15 minutes per hour).\n\nSo, the time it takes for Tony to assemble one widget, which is 5 minutes, would be:\n\n12 * 5 minutes = 60 minutes.\n\nTherefore, Tony takes 60 minutes to assemble a widget.\n</reasoning>\n<answer>\nTony takes 60 minutes to assemble a widget.\n</answer>"

In [4]:
evaluator._soft_format_reward([[{"content": completion}]])

[0.5]

In [5]:
evaluator._strict_format_reward([[{"content": completion}]])

[0.0]

In [6]:
evaluator._soft_format_reward([[{"content": completion+ "\n"}]])

[0.5]